# Exercise 1
(a) Consider the nonlinear equation:

$$
\frac{\partial u}{\partial t} + u\frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}
$$

Using:
- Central differences for the second order spatial derivative
- Upwind difference for the first order spatial derivative
- Implicit Euler for time

$$
F_i(\mathbf{U}) = \frac{U^k_i - U^{k-1}_i}{\Delta t} + U^k_i\left(\frac{U^k_i - U^k_{i-1}}{h}\right) - \nu\left(\frac{U^k_{i+1} - 2U^k_i + U^k_{i-1}}{h^2}\right)
$$

In [1]:
import numpy as np

def fun_burgers(uk, ukp, dt, h, nu, ua, ub):

  # Nonlinear residual function
  # Input: uk    -  vector U^k (dimension m x 1)
  #        ukp -  previous solution (dimension m x 1)
  #        dt   - time-step
  #        h    - spatial grid size
  #        nu    - kinematic viscosity
  #        ua    - left boundary value
  #        ub    - right boundary value
  # Output: f   - nonlinear functions evaluated at u

  m = ukp.size
  f = np.zeros( (m-2, 1) )

  # Difference equation at the left boundary
  f[0] = ( uk[0] - ukp[1] ) / dt + uk[0] * ( uk[0] - ua ) / h - nu * ( uk[1] - 2*uk[0] + ua ) / h**2 

  # Difference equations at each internal node
  for i in range(1,m-1):
    f[i] = ( uk[i] - ukp[i] ) / dt + uk[i] * ( uk[i] - uk[i-1] ) / h - nu * ( uk[i+1] - 2*uk[i] + ukp[i-1] ) / h**2 

  # Difference equation at the right boundary
  f[m-3] = ( uk[m] - ukp[m] ) / dt + uk[m] * ( uk[m] - uk[m-1] ) / h - nu * ( ub - 2*uk[m] + ukp[m-1] ) / h**2 

  return f


We can derive analytical formulas for the non zero elements of the Jacobian:

$$
\frac{\partial F_i}{\partial U_{i-1}} = -\frac{1}{h}\left(U_i -\frac{\nu}{h}\right)
$$

$$
\frac{\partial F_i}{\partial U_{i}} = \frac{1}{\Delta t} +  \left(\frac{2U_i - U_{i-1}}{h}\right) + \frac{2\nu U_i}{h^2}
$$

$$
\frac{\partial F_i}{\partial U_{i+1}} = \frac{-\nu}{h^2}
$$